# Nonnegative Matrix Factorisation for Audio Applications
## Dan Jacobellis, Tyler Masthay
![](VWH.png)

# GPU Performance
 ![](gpu_runtime.png)
 
 - Runtime is relatively flat as the number of components increases up to a point
 - We can't fit everying in GPU memory after the number of components reaches ~128
 - After this point, we are forced to send data between CPU and GPU *every* iteration
 - Fortunately, the behavior as a function of the audio length is well-behaved